# Process CCP simulation results - UTC Microgrid

In [6]:
run_from_collab = False

In [7]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle
from sys import platform

%matplotlib inline

In [8]:
# Import custom python file from github repo: https://changhsinlee.com/colab-import-python/
if run_from_collab:
    !pip install requests
    import requests
    # Save python as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url = 'https://raw.githubusercontent.com/levorato/ccp_rtcs/master/notebooks/rccp_utils.py'
    r = requests.get(url)
    # make sure your filename is the same as how you want to import 
    with open('rccp_utils.py', 'w') as f:
        f.write(r.text)
    # now we can import
    from rccp_utils import *
else:
    from rccp_utils import *

## 1. Process result files

### 1.1. Setup project folders

In [9]:
if run_from_collab:
    from google.colab import drive
    drive.mount('/content/gdrive/')
    gdrive_folder = '/content/gdrive/MyDrive'
else:
    gdrive_folder = '..'
print('gdrive_folder=', gdrive_folder)

gdrive_folder= ..


In [10]:
project_folder = '..'
antoine_instances_folder = os.path.join(project_folder, "instances", "utc_skew")
toy_instances_folder = os.path.join(project_folder, "instances", "toy")
instances_folder = os.path.join(project_folder, "instances")
japan_instances_folder = os.path.join(project_folder, "instances", "japan_microgrid")
output_folder = os.path.join(project_folder, "rccp_experiments")
print("*** Project folder is", project_folder)
print("*** Instances folder is",  instances_folder)
print("*** Output folder is", output_folder)


*** Project folder is ..
*** Instances folder is ..\instances
*** Output folder is ..\rccp_experiments


### 1.2. List which experiments to process

In [11]:
experiment_list = ["run_sim"]

In [12]:
experiment_folder_list = [os.path.join(output_folder, exp) for exp in experiment_list]
experiment_folder_list

['..\\rccp_experiments\\run_sim']

### 1.3. List which CPP models to process

In [13]:
simulated_model_list = ["robust-budget", "robust-box", "robust-budget"]

### 1.4. Select instance_group to process

In [15]:
instance_group_list = ["utc-skew"]

### 1.5. Select RTCS forecast types to process

In [16]:
forecast_type_list = ["average"]  # average-based RTCS forecast

In [17]:
#instances_to_process = ["A_instance2_1000s_skewed-left.txt", "A_instance2_1000s_skewed-right.txt", "A_instance2_1000s_uniform.txt"]
instance_group = "antoine-skew"
instances_to_process = get_instance_list(project_folder, antoine_instances_folder, toy_instances_folder, 
                                         japan_instances_folder, instance_group)
instances_to_process

Processing,  antoine-skew  instances...
# instances to process:  9


[('..\\instances\\utc_skew\\A_instance2_1000s_skewed-left.txt',
  '..\\instances\\utc_skew\\..\\instances\\utc_skew\\A_instance2_1000s_skewed-left.txt'),
 ('..\\instances\\utc_skew\\A_instance2_1000s_skewed-right.txt',
  '..\\instances\\utc_skew\\..\\instances\\utc_skew\\A_instance2_1000s_skewed-right.txt'),
 ('..\\instances\\utc_skew\\A_instance2_1000s_uniform.txt',
  '..\\instances\\utc_skew\\..\\instances\\utc_skew\\A_instance2_1000s_uniform.txt'),
 ('..\\instances\\utc_skew\\A_instance2_1NDU_Cons_1000s_skewed-left.txt',
  '..\\instances\\utc_skew\\..\\instances\\utc_skew\\A_instance2_1NDU_Cons_1000s_skewed-left.txt'),
 ('..\\instances\\utc_skew\\A_instance2_1NDU_Cons_1000s_skewed-right.txt',
  '..\\instances\\utc_skew\\..\\instances\\utc_skew\\A_instance2_1NDU_Cons_1000s_skewed-right.txt'),
 ('..\\instances\\utc_skew\\A_instance2_1NDU_Cons_1000s_uniform.txt',
  '..\\instances\\utc_skew\\..\\instances\\utc_skew\\A_instance2_1NDU_Cons_1000s_uniform.txt'),
 ('..\\instances\\utc_skew\\

### 1.6. Read consolidated result file

In [ ]:
#df_list = []
result_path = os.path.join(experiment_folder_list[0], "run_sim_utc_forecast_avg.results.pkl.gz")
df = pd.read_pickle(result_path)
#for instance_path in instances_to_process:
#    instance_name = instance_path[1][instance_path[1].rfind(os.path.sep)+1:]
#    result_path = os.path.join(experiment_folder_list[0], "output", "simulation", "zip", instance_name)
#    df_ = read_concatenated_trace_df(result_path)
#    df_['InstanceName'] = instance_name
#    df_list.append(df_)
#df = pd.concat(df_list)

In [ ]:
df.head()

In [ ]:
df.info()

### Create the output folders for processed results

In [ ]:
reportfolder = os.path.join(output_folder, 'consolidated_results')
reportfolder_graph = os.path.join(reportfolder, 'graphs')
reportfolder_table = os.path.join(reportfolder, 'tables')
if not os.path.exists(reportfolder_graph):
    os.makedirs(reportfolder_graph)
if not os.path.exists(reportfolder_table):
    os.makedirs(reportfolder_table)
print('Saving files on folder: ' + reportfolder)

## Process Tables

Obtain list of Model, Strategy, ModelPolicy, ForecastType

In [ ]:
model_list = df['Model'].unique().tolist()
strategy_list = df['Strategy'].unique().tolist()
model_policy_list = df['ModelPolicy'].unique().tolist()
reoptimize_value_list = df['Reoptimize'].unique().tolist()
forecast_type_list = df['ForecastType'].unique().tolist()
print("Model", model_list)
print("Strategy", strategy_list)
print("ModelPolicy", model_policy_list)
print("Reoptimize", reoptimize_value_list)
print("ForecastType", forecast_type_list)

## Table 1. Simulation performance given all instances 

Model-wise RTCS simulation performance comparison, given all instances.

* Median, Mean, Std. dev and sum of each measure (cost, e_td, gap, time). 

In [ ]:
per_instance_stats = dict()
for instance_name in instances_to_process:  # group by instance
    df_itype = df[(df['InstanceName'] == instance_name)]
    _forecast_type_list = df_itype['ForecastType'].unique().tolist()
    for forecast_type in _forecast_type_list:
        df_forecast = df_itype[df_itype['ForecastType'] == forecast_type]
        _model_list = df_forecast['Model'].unique().tolist()
        for model in _model_list:
            df_model = df_forecast[df_forecast['Model'] == model]
            _gamma_perc_list = df_model['GammaPerc'].unique().tolist()
            for gamma_perc in _gamma_perc_list:
                df_gamma = df_model[df_model['GammaPerc'] == gamma_perc]
                _strategy_list = df_gamma['Strategy'].unique().tolist()
                for strategy in _strategy_list:
                    df_strategy = df_gamma[df_gamma['Strategy'] == strategy]
                    _model_policy_list = df_strategy['ModelPolicy'].unique().tolist()
                    for model_policy in _model_policy_list:
                        df_policy = df_strategy[df_strategy['ModelPolicy'] == model_policy]
                        _reoptimize_value_list = df_policy['Reoptimize'].unique().tolist()
                        for reopt in _reoptimize_value_list:
                            df_reopt = df_policy[df_policy['Reoptimize'] == reopt]
                            _scenario_list = df_reopt['ScenarioId'].unique().tolist()
                            for scenario in _scenario_list:
                                df_ = df_reopt[df_reopt['ScenarioId'] == scenario]
                                key = (instance_name, forecast_type, model, gamma_perc, strategy, model_policy, reopt, scenario)
                                per_instance_stats[key] = dict()
                                #per_instance_stats[key]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
                                #per_instance_stats[key]['% Solved'] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
                                #per_instance_stats[key]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
                                per_instance_stats[key]['Median time'] = np.round(df_['RealProcTime'].median(), 2)
                                per_instance_stats[key]['Avg. time'] = np.round(df_['RealProcTime'].mean(), 2)
                                per_instance_stats[key]['Std. dev. of time'] = np.round(df_['RealProcTime'].std(), 2)
                                per_instance_stats[key]['Total time'] = np.round(df_['RealProcTime'].sum(), 2)
                                
                                per_instance_stats[key]['Median cost'] = np.round(df_['cost'].median(), 2)
                                per_instance_stats[key]['Avg. cost'] = np.round(df_['cost'].mean(), 2)
                                per_instance_stats[key]['Std. dev. of cost'] = np.round(df_['cost'].std(), 2)
                                per_instance_stats[key]['Total cost'] = np.round(df_['cost'].sum(), 2)
                                
                                per_instance_stats[key]['Median gap'] = np.round(df_['gap'].median(), 2)
                                per_instance_stats[key]['Avg. gap'] = np.round(df_['gap'].mean(), 2)
                                per_instance_stats[key]['Std. dev. of gap'] = np.round(df_['gap'].std(), 2)
                                per_instance_stats[key]['Total gap'] = np.round(df_['gap'].sum(), 2)
                                
                                per_instance_stats[key]['Median e_td'] = np.round(df_['e_td'].median(), 2)
                                per_instance_stats[key]['Avg. e_td'] = np.round(df_['e_td'].mean(), 2)
                                per_instance_stats[key]['Std. dev. of e_td'] = np.round(df_['e_td'].std(), 2)
                                per_instance_stats[key]['Total e_td'] = np.round(df_['e_td'].sum(), 2)

In [ ]:
df_table1 = pd.DataFrame.from_dict(per_instance_stats)
df_table1

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### Table 1. Total cost of each simulation, grouped by CCP model and simulation parameters

In [ ]:
df.drop(columns=['t', 'd', 'OptTimeSpent']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType']).sum()

### Table 2. Cost of the most expensive scenario (worst simulation cost), grouped by CCP model and simulation parameters

In [ ]:
df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType', 'ScenarioId']).sum().\
    groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType']).max()

### Table 3. RTCS performance map (robust wins)

Number of scenarios where Robust RTCS obtained smaller cost, when compared to the Deterministic RTCS, when simulating the same scenario.

In [ ]:
df_group = df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType', 'ScenarioId']).sum()\
    .drop(columns=['gap', 'RealProcTime'])
df_rob = df_group.reset_index()
df_rob = df_rob[(df_rob['Model'] == 'robust-budget') | (df_rob['Model'] == 'robust-box')]
df_det = df_group.reset_index()
df_det = df_det[df_det['Model'] == 'deterministic']
#df_rob
df_wins = df_rob.merge(df_det, on=['InstanceName', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType', 'ScenarioId'], suffixes=('_rob', '_det'))\
    .drop(columns=['Model_det', 'Gamma_rob', 'Gamma_det', 'GammaPerc_det'])

In [ ]:
df_wins['rob_wins'] = (df_wins['cost_rob'] < df_wins['cost_det']).astype(int)

In [ ]:
df_wins.head(10)

In [ ]:
df_wins.groupby(by=['InstanceName', 'Model_rob', 'GammaPerc_rob', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType']).sum().drop(columns=['ScenarioId'])

### Table 4. Cheapest RTCS Strategy, per instance and model type

In [ ]:
df_group = df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType', 'ScenarioId']).sum()\
    .drop(columns=['gap', 'RealProcTime'])
# Find the cheapest strategy for each model type
df_cheapest = df_group.groupby(by=['InstanceName', 'Model', 'GammaPerc', 'ScenarioId']).min().drop(columns=['e_td'])

In [ ]:
df_rob = df_cheapest.reset_index()
df_rob = df_rob[(df_rob['Model'] == 'robust-budget') | (df_rob['Model'] == 'robust-box')]
df_det = df_cheapest.reset_index()
df_det = df_det[df_det['Model'] == 'deterministic']
df_wins = df_rob.merge(df_det, on=['InstanceName', 'ScenarioId'], suffixes=('_rob', '_det'))\
    .drop(columns=['Model_det', 'Gamma_rob', 'Gamma_det', 'GammaPerc_det'])
df_wins['rob_wins'] = (df_wins['cost_rob'] <= df_wins['cost_det']).astype(int)
df_wins['det_wins'] = (df_wins['cost_rob'] > df_wins['cost_det']).astype(int)

In [ ]:
df_wins.groupby(by=['InstanceName', 'Model_rob', 'GammaPerc_rob']).sum().drop(columns=['ScenarioId'])

### TODO Fazer um kde distribution plot dos custos do RTCS obtidos nas simulacoes: robusto-gamma vs. deterministico

### TODO Fazer uma tabela com as medidas estatisticas (para cada distribuicao usada) de cada simulacao, incluindo valor esperado, SD, percentis 95, 99 e valor maximo observado empiricamente.

In [ ]:
for experiment_folder in experiment_folder_list:
    for instance_group in instance_group_list:
        instance_list = get_instance_list(project_folder, antoine_instances_folder, toy_instances_folder, instance_group)
        print(instance_group, instance_list)
        for model in simulated_model_list:
            for forecast_type in forecast_type_list:
                print(model, forecast_type)